<a href="https://colab.research.google.com/github/krishna663-wq/Neural_Networks/blob/main/Hyperparameter_Tuning_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hyper parameter Tuning**

1. No. of Neurons in a hidden layer.
2. No. of hidden layers.
3. Activation function in Hidden layer.
4. Dropout 1000


In [1]:
import pandas as pd
data = pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
data.shape

(768, 9)

In [8]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Data is ready to feed into the NN.

In [16]:
!pip install keras-tuner

In [18]:
import kerastuner as kt
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [42]:
def build_model(hp):
  model = Sequential()
  model.add(
            Dense(hp.Int("unit_0",min_value=8,max_value=128,step=8),
            activation=hp.Choice("activation_0",values=["sigmoid","relu","tanh"]),
            input_dim=8)
            )
  model.add(Dropout(hp.Choice("dropout_0",values=[0.10,0.20,0.30,0.40])))
  for i in range(1,hp.Int("num_layers",min_value=1,max_value=10)):
    model.add(
            Dense(hp.Int(f"unit_{i}",min_value=8,max_value=128,step=8),
            activation=hp.Choice(f"activation_{i}",values=["sigmoid","relu","tanh"]))
            )
    # Adding dropout layer within the loop, so 'i' is defined
    model.add(Dropout(hp.Choice(f"dropout_{i}",values=[0.10,0.20,0.30,0.40])))

  model.add(Dense(1,activation='sigmoid'))
  # Include 'accuracy' in the metrics to calculate and report it
  model.compile(loss='binary_crossentropy',optimizer=hp.Choice("optimizer", values=['adam','adagrad','rmsprop','adadelta']), metrics=['accuracy'])
  return model

In [43]:
tuner = kt.RandomSearch(build_model,
                objective='val_accuracy',
                max_trials=3,
                directory = "diabetes",
                project_name = "diabetes_tuning")

Reloading Tuner from diabetes/diabetes_tuning/tuner0.json


In [44]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.6948052048683167
Total elapsed time: 00h 02m 34s


In [45]:
tuner.get_best_hyperparameters()[0].values

{'unit_0': 16,
 'activation_0': 'relu',
 'dropout_0': 0.4,
 'num_layers': 5,
 'optimizer': 'adagrad',
 'unit_1': 16,
 'activation_1': 'relu',
 'dropout_1': 0.4,
 'unit_2': 32,
 'activation_2': 'relu',
 'dropout_2': 0.4,
 'unit_3': 112,
 'activation_3': 'relu',
 'dropout_3': 0.1,
 'unit_4': 8,
 'activation_4': 'sigmoid',
 'dropout_4': 0.4,
 'unit_5': 64,
 'activation_5': 'relu',
 'dropout_5': 0.4,
 'unit_6': 128,
 'activation_6': 'tanh',
 'dropout_6': 0.3}

In [46]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [47]:
model.fit(X_train,y_train,initial_epoch = 11,epochs=100,validation_data=(X_test,y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.5366 - loss: 0.7061 - val_accuracy: 0.6948 - val_loss: 0.6665
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5269 - loss: 0.7101 - val_accuracy: 0.6948 - val_loss: 0.6648
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5087 - loss: 0.7254 - val_accuracy: 0.6948 - val_loss: 0.6636
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5115 - loss: 0.7183 - val_accuracy: 0.6948 - val_loss: 0.6626
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5277 - loss: 0.7187 - val_accuracy: 0.6948 - val_loss: 0.6616
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5599 - loss: 0.7022 - val_accuracy: 0.6948 - val_loss: 0.6604
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5210 - loss: 0.7197 - val_accuracy: 0.6948 - val_loss: 0.6596
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5263 - loss: 0.7011 - val_accuracy